# Añadir y eliminar datos

## Acerca de los datos
En este cuaderno trabajaremos con datos de terremotos del 18 de septiembre de 2018 al 13 de octubre de 2018 (obtenidos del US Geological Survey (USGS) mediante la [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Configuración
Estaremos trabajando con el archivo `data/earthquakes.csv` nuevamente, por lo que necesitamos manejar nuestras importaciones y leerlo.

In [ ]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Creación de nuevos datos
### Añadir nuevas columnas
Las nuevas columnas se añaden a la derecha de las columnas originales y pueden ser un único valor, que será **difundido** a lo largo de las filas del marco de datos:

In [ ]:
df['source'] = 'USGS API'
df.head()

...o una máscara booleana:

In [ ]:
df['mag_negative'] = df.mag < 0
df.head()

#### Añadir la columna `parsed_place`
Tenemos un problema de reconocimiento de entidades con la columna `place`. Hay varias entidades que tienen varios nombres en los datos (por ejemplo, CA y California, NV y Nevada).

In [ ]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

Reemplazar partes de los nombres `place` para adaptarlos a nuestras necesidades:

In [ ]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True # eliminar el "of"
).str.replace(
    'the ', '' # eliminar "the"
).str.replace(
    r'CA$', 'California', regex=True # Corregir California
).str.replace(
    r'NV$', 'Nevada', regex=True # Corregir Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True # Corregir Mexico
).str.replace(
    r' region$', '', regex=True # cortar las terminaciones con "región"
).str.replace(
    'northern ', '' # eliminar "northern"
).str.replace(
    'Fiji Islands', 'Fiji' # Alinear las plazas de Fiji
).str.replace(
    r'^.*, ', '', regex=True # eliminar cualquier otra cosa extraña desde el principio
).str.strip() # eliminar los espacios sobrantes

Ahora podemos utilizar un solo nombre para obtener todos los terremotos de ese lugar (aunque esto todavía no es perfecto):

In [ ]:
df.parsed_place.sort_values().unique()

#### Utilización del método `assign()` para crear columnas
Para crear muchas columnas a la vez o actualizar columnas existentes, podemos utilizar `assign()`:

In [ ]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

Con el uso de funciones `lambda`, el método `assign()` se vuelve aún más potente. **Las funciones lambda** son funciones anónimas que suelen definirse en una sola línea y para un solo uso. El método `assign()` pasa todo el marco de datos a la función `lambda` como `x`; desde ahí, podemos seleccionar las columnas `in_ca` y `in_alaska`, que se están creando en esa misma llamada a `assign()`. Aquí, utilizamos una función `lambda` para crear una nueva columna, `neither`, que indica si el terremoto no se produjo ni en Alaska ni en California:

In [ ]:
df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

#### Concatenación
Supongamos que trabajamos con dos marcos de datos distintos, uno con terremotos acompañados de tsunamis y otro con terremotos sin tsunamis. Si quisiéramos ver los terremotos en su conjunto, tendríamos que concatenar los marcos de datos en uno solo:

In [ ]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

Concatenar a lo largo del eje de filas (`axis=0`) equivale a añadir hasta el final. Al concatenar los terremotos con tsunami y sin tsunami, obtenemos el conjunto completo de datos de terremotos:

In [ ]:
pd.concat([tsunami, no_tsunami]).shape

Observe que el resultado anterior es equivalente a ejecutar el método `append()` del marco de datos:

In [ ]:
pd.concat([tsunami, no_tsunami]).shape

Hemos estado trabajando con un subconjunto de las columnas del fichero CSV, pero supongamos que ahora queremos obtener algunas de las columnas que ignoramos cuando leímos los datos. Como hemos añadido nuevas columnas en este cuaderno, no querremos leer el fichero y volver a realizar esas operaciones. En su lugar, concatenaremos a lo largo de las columnas (`axis=1`) para volver a añadir lo que nos falta:

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

Pero fíjate en lo que ocurre si el índice no está alineado:

In [ ]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

Si el índice no está alineado, podemos alinearlo antes de intentar la concatenación, de la que hablaremos en el capítulo 3.

Digamos que queremos unir los marcos de datos `tsunami` y `no_tsunami`, pero el marco de datos `no_tsunami` tiene una columna adicional. El parámetro `join` especifica cómo manejar cualquier solapamiento en los nombres de las columnas (cuando se añaden a la parte inferior) o en los nombres de las filas (cuando se concatenan a la izquierda/derecha). Por defecto, este parámetro es `outer`, por lo que conservamos todo; sin embargo, si utilizamos `inner`, sólo conservaremos lo que esté en común:

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

Además, utilizamos `ignore_index`, ya que el índice no significa nada para nosotros aquí. Esto nos da valores secuenciales en lugar de lo que teníamos en el resultado anterior:

In [ ]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

## Borrado de datos no deseados
Las columnas pueden borrarse utilizando la sintaxis de diccionario con `del`:

In [ ]:
del df['source']
df.columns

Si no sabemos si la columna existe, debemos utilizar un bloque `try`/`except`:

In [ ]:
try:
    del df['source']
except KeyError:
    # handle the error here
    print('not there anymore')

También podemos utilizar `pop()`. Esto nos permitirá utilizar las series que eliminemos más tarde. Tenga en cuenta que habrá un error si la clave no existe, por lo que también podemos utilizar un `try`/`except` aquí:

In [ ]:
mag_negative = df.pop('mag_negative')
df.columns

Fíjate que ahora tenemos una máscara en `mag_negative`:

In [ ]:
mag_negative.value_counts()

Ahora, podemos utilizar `mag_negative` para filtrar nuestros datos:

In [ ]:
df[mag_negative].head()

### Usando el método `drop()`.
Podemos eliminar filas pasando una lista de índices al método `drop()`. Observa en el siguiente ejemplo que cuando pedimos las 2 primeras filas con `head()` obtenemos la 3ª y 4ª filas porque hemos eliminado las 2 primeras originales con `drop([0, 1])`:

In [ ]:
df.drop([0, 1]).head(2)

El método `drop()` elimina por defecto a lo largo del eje de filas. Si pasamos una lista de columnas con el argumento `columns`, podemos eliminar columnas:

In [ ]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

También tenemos la opción de utilizar `axis=1`:

In [ ]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

Por defecto, `drop()`, junto con la mayoría de los métodos `DataFrame`, devolverá un nuevo objeto `DataFrame`. Si sólo queremos cambiar el objeto con el que estamos trabajando, podemos pasarle `inplace=True`. Esto debe usarse con cuidado:

In [ ]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="text-align: left;">
        <a href="./5-subconjunto_data.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="text-align: center;">
        <a href="../solutions/ch_02/solutions.ipynb">
            <button>Soluciones</button>
        </a>
    </div>
    <div style="text-align: right;">
        <a href="../ch_03/1-ancho_vs_largo.ipynb">
            <button>Capitulo 3 &#8594;</button>
        </a>
    </div>
</div>

<hr>
